In [ ]:
import numpy as np
import cv2
import re
from keras.models import model_from_json, Model
import matplotlib.pyplot as plt
from grad_cam import make_gradcam_heatmap, superimpose
from utils import get_pred_class, sample_from_true_prediction_given_class, get_layer_index_by_name

import tensorflow as tf
print("Num GPUs Available:", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

## Loading data

In [ ]:
x_data = np.load('../data/soja_images_150_new.npy', allow_pickle=True)
y_data = np.load('../data/soja_labels_150_new.npy', allow_pickle=True)

x_train = np.load('../data/x_train.npy', allow_pickle=True)
x_test = np.load('../data/x_test.npy', allow_pickle=True)
y_train = np.load('../data/y_train.npy', allow_pickle=True)
y_test = np.load('../data/y_test.npy', allow_pickle=True)

out_size = y_train.shape[1]

## Distribution of response variable

- 0: normal
- 1: quebrado
- 2: avariado
- 3: verde
- 4: ardido

In [ ]:
plt.figure(figsize=(8, 5))
plt.bar(np.unique(y_data), np.bincount(y_data))
plt.xlabel("Class")
plt.ylabel("Count")
plt.title("Distribution of soybean classes")
# plt.savefig("../images/soybean_count.png")

In [ ]:
seed = 1932

n_images = 6

# fig, axs = plt.subplots(out_size, n_images, figsize=(8, 8))
fig, axs = plt.subplots(out_size, n_images, figsize=(16, 12))

for i in range(out_size):
    np.random.seed(seed)
    sample_id = np.random.choice(np.where(y_data==i)[0], n_images)
    for j in range(n_images):
        axs[i, j].imshow(x_data[sample_id[j]])
        axs[i, j].set_yticklabels([])
        axs[i, j].set_xticklabels([])
        axs[i, j].set_title(i)

# plt.savefig("../images/soybean_grid.png")

## Loading trained model

In [ ]:
import mlflow
kk = mlflow.keras.load_model("./mlruns/0/710cb4e3e4a84168bde8b621d6b8c798/artifacts/models/")
kk.evaluate(x_test, y_test)

In [ ]:
with open("../data/model2.json", "r") as f:
    model_json = f.read()

model = model_from_json(model_json)
model.load_weights("../data/weights2.h5")

In [ ]:
model.summary()

## Grad-CAM

In [ ]:
img_size = x_train.shape[:2]

last_conv_layer_name = "conv5_block3_out"
classifier_layer_names = [
    "out_pool",
    "out_layer"
]

# Get labels
y_class = np.where(y_test == 1)[1]

# Get predicted labels
pred_class = get_pred_class(model, x_test)

In [ ]:
seed = 329
n_images = 6
alpha = 0.5

fig, axs = plt.subplots(out_size, n_images, figsize=(16,12))

for i in range(out_size):
    sample_id = sample_from_true_prediction_given_class(pred_class, y_class, n_images, i, seed)
    for j in range(n_images):
        img = x_test[sample_id[j]]
        img_model = np.expand_dims(img, axis=0)
        heatmap = make_gradcam_heatmap(img_model, model, last_conv_layer_name, classifier_layer_names)
        superimp = superimpose(img, heatmap, alpha)
        axs[i, j].imshow(superimp)
        axs[i, j].set_yticklabels([])
        axs[i, j].set_xticklabels([])
        axs[i, j].set_title(i)

### Visualizing per channel color

In [ ]:
seed = 352
n_channels = 3
channels_name = ["Red", "Green", "Blue"]
n_images = 5
alpha = 0.8

class_value = 0
sample_id = sample_from_true_prediction_given_class(pred_class, y_class, n_images, class_value, seed)

fig, axs = plt.subplots(n_images, n_channels + 2, figsize=(14,12))

for i in range(n_images):
    img = x_test[sample_id[i]]
    img_model = np.expand_dims(img, axis=0)
    heatmap = make_gradcam_heatmap(img_model, model, last_conv_layer_name, classifier_layer_names)
    superimp = superimpose(img, heatmap, alpha)

    axs[i, 0].imshow(img.astype(np.uint8))
    axs[i, 0].set_yticklabels([])
    axs[i, 0].set_xticklabels([])
    axs[i, 0].set_title("Original")
    
    axs[i, 1].imshow(superimp)
    axs[i, 1].set_yticklabels([])
    axs[i, 1].set_xticklabels([])
    axs[i, 1].set_title("Grad-CAM")
        
    for j in range(n_channels):
        img_channel = img[:, :, j]
        img_channel = cv2.cvtColor(img_channel, cv2.COLOR_GRAY2RGB).astype(np.uint8)
        axs_pos = j + 2
        axs[i, axs_pos].imshow(img_channel)
        axs[i, axs_pos].set_yticklabels([])
        axs[i, axs_pos].set_xticklabels([])
        axs[i, axs_pos].set_title(channels_name[j])

In [ ]:
seed = 352
n_channels = 3
channels_name = ["Red", "Green", "Blue"]
n_images = 5
alpha = 0.8

class_value = 1
sample_id = sample_from_true_prediction_given_class(pred_class, y_class, n_images, class_value, seed)

fig, axs = plt.subplots(n_images, n_channels + 2, figsize=(14,12))

for i in range(n_images):
    img = x_test[sample_id[i]]
    img_model = np.expand_dims(img, axis=0)
    heatmap = make_gradcam_heatmap(img_model, model, last_conv_layer_name, classifier_layer_names)
    superimp = superimpose(img, heatmap, alpha)

    axs[i, 0].imshow(img.astype(np.uint8))
    axs[i, 0].set_yticklabels([])
    axs[i, 0].set_xticklabels([])
    axs[i, 0].set_title("Original")
    
    axs[i, 1].imshow(superimp)
    axs[i, 1].set_yticklabels([])
    axs[i, 1].set_xticklabels([])
    axs[i, 1].set_title("Grad-CAM")
        
    for j in range(n_channels):
        img_channel = img[:, :, j]
        img_channel = cv2.cvtColor(img_channel, cv2.COLOR_GRAY2RGB).astype(np.uint8)
        axs_pos = j + 2
        axs[i, axs_pos].imshow(img_channel)
        axs[i, axs_pos].set_yticklabels([])
        axs[i, axs_pos].set_xticklabels([])
        axs[i, axs_pos].set_title(channels_name[j])

In [ ]:
seed = 352
n_channels = 3
channels_name = ["Red", "Green", "Blue"]
n_images = 5
alpha = 0.8

class_value = 2
sample_id = sample_from_true_prediction_given_class(pred_class, y_class, n_images, class_value, seed)

fig, axs = plt.subplots(n_images, n_channels + 2, figsize=(14,12))

for i in range(n_images):
    img = x_test[sample_id[i]]
    img_model = np.expand_dims(img, axis=0)
    heatmap = make_gradcam_heatmap(img_model, model, last_conv_layer_name, classifier_layer_names)
    superimp = superimpose(img, heatmap, alpha)

    axs[i, 0].imshow(img.astype(np.uint8))
    axs[i, 0].set_yticklabels([])
    axs[i, 0].set_xticklabels([])
    axs[i, 0].set_title("Original")
    
    axs[i, 1].imshow(superimp)
    axs[i, 1].set_yticklabels([])
    axs[i, 1].set_xticklabels([])
    axs[i, 1].set_title("Grad-CAM")
        
    for j in range(n_channels):
        img_channel = img[:, :, j]
        img_channel = cv2.cvtColor(img_channel, cv2.COLOR_GRAY2RGB).astype(np.uint8)
        axs_pos = j + 2
        axs[i, axs_pos].imshow(img_channel)
        axs[i, axs_pos].set_yticklabels([])
        axs[i, axs_pos].set_xticklabels([])
        axs[i, axs_pos].set_title(channels_name[j])

In [ ]:
seed = 352
n_channels = 3
channels_name = ["Red", "Green", "Blue"]
n_images = 5
alpha = 0.8

class_value = 3
sample_id = sample_from_true_prediction_given_class(pred_class, y_class, n_images, class_value, seed)

fig, axs = plt.subplots(n_images, n_channels + 2, figsize=(14,12))

for i in range(n_images):
    img = x_test[sample_id[i]]
    img_model = np.expand_dims(img, axis=0)
    heatmap = make_gradcam_heatmap(img_model, model, last_conv_layer_name, classifier_layer_names)
    superimp = superimpose(img, heatmap, alpha)

    axs[i, 0].imshow(img.astype(np.uint8))
    axs[i, 0].set_yticklabels([])
    axs[i, 0].set_xticklabels([])
    axs[i, 0].set_title("Original")
    
    axs[i, 1].imshow(superimp)
    axs[i, 1].set_yticklabels([])
    axs[i, 1].set_xticklabels([])
    axs[i, 1].set_title("Grad-CAM")
        
    for j in range(n_channels):
        img_channel = img[:, :, j]
        img_channel = cv2.cvtColor(img_channel, cv2.COLOR_GRAY2RGB).astype(np.uint8)
        axs_pos = j + 2
        axs[i, axs_pos].imshow(img_channel)
        axs[i, axs_pos].set_yticklabels([])
        axs[i, axs_pos].set_xticklabels([])
        axs[i, axs_pos].set_title(channels_name[j])

In [ ]:
seed = 352
n_channels = 3
channels_name = ["Red", "Green", "Blue"]
n_images = 5
alpha = 0.8

class_value = 4
sample_id = sample_from_true_prediction_given_class(pred_class, y_class, n_images, class_value, seed)

fig, axs = plt.subplots(n_images, n_channels + 2, figsize=(14,12))

for i in range(n_images):
    img = x_test[sample_id[i]]
    img_model = np.expand_dims(img, axis=0)
    heatmap = make_gradcam_heatmap(img_model, model, last_conv_layer_name, classifier_layer_names)
    superimp = superimpose(img, heatmap, alpha)

    axs[i, 0].imshow(img.astype(np.uint8))
    axs[i, 0].set_yticklabels([])
    axs[i, 0].set_xticklabels([])
    axs[i, 0].set_title("Original")
    
    axs[i, 1].imshow(superimp)
    axs[i, 1].set_yticklabels([])
    axs[i, 1].set_xticklabels([])
    axs[i, 1].set_title("Grad-CAM")
        
    for j in range(n_channels):
        img_channel = img[:, :, j]
        img_channel = cv2.cvtColor(img_channel, cv2.COLOR_GRAY2RGB).astype(np.uint8)
        axs_pos = j + 2
        axs[i, axs_pos].imshow(img_channel)
        axs[i, axs_pos].set_yticklabels([])
        axs[i, axs_pos].set_xticklabels([])
        axs[i, axs_pos].set_title(channels_name[j])

## Layers Activations

In [ ]:
layer_idx = get_layer_index_by_name(model, "out_pool")

layer_names = [layer.name for layer in model.layers[:layer_idx]]
layer_outputs = [layer.output for layer in model.layers[:layer_idx]] 

activation_model = Model(inputs=model.input, outputs=layer_outputs)

### Selecting layers for plot

In [ ]:
# pattern = re.compile("input_1|\w+_bn$|\w+_pad$")

# layer_names_selected = []
# for name in layer_names:
#     if not bool(pattern.match(name)):
#         layer_names_selected.append(name)

layer_names_selected = ["pool1_pool", "conv2_block3_out", "conv3_block4_out", "conv4_block6_out", 
                        "conv5_block3_out"]

In [ ]:
img = x_test[52]
img_model = np.expand_dims(img, axis=0)

activations = activation_model.predict(img_model)

In [ ]:
images_per_row = 16
max_imgs = images_per_row * 4

for layer_name, layer_activation in zip(layer_names, activations): # Displays the feature maps    
    if layer_name in layer_names_selected:
        n_features = layer_activation.shape[-1] # Number of features in the feature map
        if n_features > max_imgs:
            n_features = max_imgs
        size = layer_activation.shape[1] #The feature map has shape (1, size, size, n_features).
        n_cols = n_features // images_per_row # Tiles the activation channels in this matrix
        display_grid = np.zeros((size * n_cols, images_per_row * size))
        for col in range(n_cols): # Tiles each filter into a big horizontal grid
            for row in range(images_per_row):
                channel_image = layer_activation[0, :, :, col * images_per_row + row]
                channel_image = (channel_image - channel_image.mean()) / channel_image.std()
                channel_image *= 64
                channel_image += 128
                channel_image = np.clip(channel_image, 0, 255).astype('uint8')
                display_grid[col * size : (col + 1) * size, # Displays the grid
                             row * size : (row + 1) * size] = channel_image
        scale = 1. / size
        plt.figure(figsize=(scale * display_grid.shape[1], scale * display_grid.shape[0]))
        plt.title(layer_name)
        plt.grid(False)
        plt.axis("off")
        plt.imshow(display_grid, aspect='auto', cmap='viridis')